In [1]:
pip install nselib

In [2]:
from nselib import capital_market
d=capital_market.index_data(index='NIFTY 50',from_date='01-01-2019',to_date='31-12-2023')
d.to_csv('output_data4.csv', index=False)

In [4]:
import csv
from datetime import datetime

#  date string to dd/mm/yyyy format
def parse_date(date_str):
    return datetime.strptime(date_str, '%d-%m-%Y')

# converting data into a list of dictionaries
with open('output_data4.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row for row in reader]

sorted_data = sorted(data, key=lambda x: parse_date(x['TIMESTAMP']))

# Write sorted data back to a CSV file
with open('sorted_output.csv', 'w', newline='') as csvfile:
    fieldnames = sorted_data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in sorted_data:
        writer.writerow(row)

In [8]:
import pandas as pd
data_df = pd.read_csv('sorted_output.csv')
data_df = data_df.rename(columns={'TIMESTAMP': 'DATE'})
data_df

,DATE,INDEX_NAME,OPEN_INDEX_VAL,HIGH_INDEX_VAL,CLOSE_INDEX_VAL,LOW_INDEX_VAL,TRADED_QTY,TURN_OVER
0,01-01-2019,NIFTY 50,10881.70,10923.60,10910.10,10807.10,159404542,8688.26
1,02-01-2019,NIFTY 50,10868.85,10895.35,10792.50,10735.05,309665939,15352.25
2,03-01-2019,NIFTY 50,10796.80,10814.05,10672.25,10661.25,286241745,15030.45
3,04-01-2019,NIFTY 50,10699.70,10741.05,10727.35,10628.65,296596655,14516.74
4,07-01-2019,NIFTY 50,10804.85,10835.95,10771.80,10750.15,269371080,12731.29
...,...,...,...,...,...,...,...,...
1239,22-12-2023,NIFTY 50,21295.85,21390.50,21349.40,21232.45,284708918,27709.29
1240,26-12-2023,NIFTY 50,21365.20,21477.15,21441.35,21329.45,219467748,20081.33
1241,27-12-2023,NIFTY 50,21497.65,21675.75,21654.75,21495.80,256542963,23059.25
1242,28-12-2023,NIFTY 50,21715.00,21801.45,21778.70,21678.00,393080755,35031.00


In [9]:
from nselib import derivatives


In [10]:
#Sampe demostration of how to calculate pcr ratio
derivatives.participant_wise_open_interest('02-02-2024')

,Client Type,Future Index Long,Future Index Short,Future Stock Long,Future Stock Short,Option Index Call Long,Option Index Put Long,Option Index Call Short,Option Index Put Short,Option Stock Call Long,Option Stock Put Long,Option Stock Call Short,Option Stock Put Short,Total Long Contracts,Total Short Contracts
0,Client,287067,197491,1789090,232168,4731095,3487206,4459857,3664646,1169027,475354,759498,630064,11938838,9943724
1,DII,56219,90258,69698,2212014,0,24754,0,0,21,40,93171,0,150732,2395443
2,FII,69557,144454,1630414,1309189,956755,1050207,1028894,1031466,71504,66350,97469,62495,3844787,3673967
3,Pro,64490,45130,453044,188875,1436374,1372242,1635473,1238297,410854,480845,701268,330030,4217849,4139073


In [11]:
data = derivatives.participant_wise_open_interest('02-02-2024')
call = data['Option Index Call Short'].sum()
put = data['Option Index Put Short'].sum()
pcr_ratio=round(put/call, 2)
print(pcr_ratio)

0.83


In [12]:

def calculate_pcr_ratio(date):
    try:

        data = derivatives.participant_wise_open_interest(date)


        pcr_ratio = data['Option Index Put Short'].sum() / data['Option Index Call Short'].sum()

        return round(pcr_ratio, 2)  # Round to two decimal points
    except Exception as e:
        print(f"Error calculating PCR ratio for {date}: {e}")
        return None


data_df['PCR Ratio'] = data_df['DATE'].apply(calculate_pcr_ratio)


Error calculating PCR ratio for 12-12-2019: unsupported operand type(s) for /: 'str' and 'str'
Error calculating PCR ratio for 31-10-2023:  No data available for : 2023-10-31 00:00:00


In [ ]:
data_df['CLOSE_PRICE'] = data_df['CLOSE_PRICE'].round()
data_df = data_df[data_df['PCR Ratio'].notna()]
data_df.to_csv('output_data10.csv', index=False)
print(data_df)

            DATE  OPEN_INDEX_VAL  HIGH_INDEX_VAL  CLOSE_PRICE  LOW_INDEX_VAL  \
0     01-01-2019        10881.70        10923.60      10910.0       10807.10   
1     02-01-2019        10868.85        10895.35      10792.0       10735.05   
2     03-01-2019        10796.80        10814.05      10672.0       10661.25   
3     04-01-2019        10699.70        10741.05      10727.0       10628.65   
4     07-01-2019        10804.85        10835.95      10772.0       10750.15   
...          ...             ...             ...          ...            ...   
1235  22-12-2023        21295.85        21390.50      21349.0       21232.45   
1236  26-12-2023        21365.20        21477.15      21441.0       21329.45   
1237  27-12-2023        21497.65        21675.75      21655.0       21495.80   
1238  28-12-2023        21715.00        21801.45      21779.0       21678.00   
1239  29-12-2023        21737.65        21770.30      21731.0       21676.90   

      PCR Ratio  
0          1.36  
1  